In [84]:
import string
import nltk
import re
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from IPython.core.display import clear_output
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.tag import pos_tag_sents
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfTransformer
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor

/var/folders/qr/c62bp79j3c91gdkr6wydmq7h0000gn/T/ipykernel_35680/617071231.py:9: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


In [112]:
articles = pd.read_csv( "full_article2.txt", index_col=None, header=None, engine='python')
articles.columns = ['Symbol', 'Article', 'Month-Day', 'Year']
articles['Year'] = articles['Year'].astype(str)
articles['Date'] = articles['Month-Day'] + " " + articles['Year']
articles['Date'] = articles['Date'].str.strip()
articles['Date'] = articles['Date'].str.slice(start = 2, stop = 13)
articles['Date'] = articles['Date'].str.strip()
articles = articles[['Symbol', 'Article', 'Date']]
articles

,Symbol,Article,Date
0,MO,The other big story Monday was oil storming h...,Mar 22 2022
1,MO,By Peter NurseInvesting.com -- Stocks in focu...,Mar 22 2022
2,MO,The members of this alcohol and tobacco indus...,Mar 17 2022
3,MO,(Reuters) - An administrative law judge dismi...,Feb 24 2022
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,Feb 15 2022
...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,Jun 25 2015
62475,ZTS,Investing.com – U.S. stocks were lower after ...,Jun 25 2015
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,Apr 27 2015
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,Mar 09 2015


In [114]:
articles.at[articles[articles['Date'].str.contains('hours') == True].index, 'Date'] = 'Apr 06 2022'
articles = articles[articles['Date'].str.len() == 11]
articles = articles[articles['Date'].str.contains("minutes") == False]
articles['Date'] = pd.to_datetime(articles['Date'], format = '%b %d %Y')
articles

/Users/ryanshintaku/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/ryanshintaku/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,Symbol,Article,Date
0,MO,The other big story Monday was oil storming h...,2022-03-22
1,MO,By Peter NurseInvesting.com -- Stocks in focu...,2022-03-22
2,MO,The members of this alcohol and tobacco indus...,2022-03-17
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15
...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,2015-06-25
62475,ZTS,Investing.com – U.S. stocks were lower after ...,2015-06-25
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,2015-04-27
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,2015-03-09


In [115]:
df = articles.copy()
df

,Symbol,Article,Date
0,MO,The other big story Monday was oil storming h...,2022-03-22
1,MO,By Peter NurseInvesting.com -- Stocks in focu...,2022-03-22
2,MO,The members of this alcohol and tobacco indus...,2022-03-17
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15
...,...,...,...
62474,ZTS,(Reuters) - Canada-based drugmaker Valeant Ph...,2015-06-25
62475,ZTS,Investing.com – U.S. stocks were lower after ...,2015-06-25
62476,ZTS,By David RandallNEW YORK (Reuters) - When the...,2015-04-27
62477,ZTS,By Svea Herbst-BaylissBOSTON (Reuters) - Bill...,2015-03-09


In [116]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    sentence = []
    for word, tag in tokens:
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v' 
        else:
            pos = 'a'
        sentence.append(lemmatizer.lemmatize(word, pos))
    return sentence

In [118]:
df['title/no_stopwords'] = df.apply(lambda row: pos_tag(word_tokenize(row['Article'])), axis = 1)
df['title/no_stopwords'] = df.apply(lambda row: lemmatize_sentence(row['title/no_stopwords']), axis = 1)
df

KeyboardInterrupt: 

In [7]:
def remove_noise(reviews, stop_words = ()):

    cleaned_tokens = []

    for token in reviews:
        token = token.strip()
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        token = token.replace('--', '')
        token = token.replace('-', '')
        token = token.replace('–', '')
        token = token.replace('investing.com', '')
        token = token.replace('``', '')
        token = token.replace("'s", '')
        token = token.replace('..', '')
        token = token.replace('...', '')
        token = token.replace("'", '')
        token = token.replace("’",'')

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [8]:
stop_words = list(stopwords.words('english'))
df['title/no_stopwords'] = df.apply(lambda row: remove_noise(row['title/no_stopwords'], stop_words), axis = 1)
df

,Symbol,Article,Date,title/no_stopwords
0,MO,The other big story Monday was oil storming h...,2022-03-22,"[big, story, monday, oil, storm, high, 6, decl..."
1,MO,By Peter NurseInvesting.com -- Stocks in focu...,2022-03-22,"[peter, nurseinvesting.com, stocks, focus, pre..."
2,MO,The members of this alcohol and tobacco indus...,2022-03-17,"[member, alcohol, tobacco, industrybacked, mar..."
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24,"[reuters, administrative, law, judge, dismiss,..."
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15,"[washington, reuters, judge, u.s., federal, tr..."
...,...,...,...,...
6365,AAPL,By Stephen Nellis(Reuters) - If Nvidia Corp ...,2020-10-07,"[stephen, nellis, reuters, nvidia, corp, nvda,..."
6366,AAPL,By Yasin EbrahimInvesting.com – The Dow ralli...,2020-10-07,"[yasin, ebrahiminvesting.com, dow, rally, wedn..."
6367,AAPL,By Geoffrey Smith Investing.com -- Donald Tru...,2020-10-06,"[geoffrey, smith, investing.com, donald, trump..."
6368,AAPL,By Sruthi Shankar(Reuters) - European stocks ...,2020-10-06,"[sruthi, shankar, reuters, european, stock, ea..."


In [9]:
df['title/clean'] = df.apply(lambda row: ' '.join(row['title/no_stopwords']), axis = 1)
df

,Symbol,Article,Date,title/no_stopwords,title/clean
0,MO,The other big story Monday was oil storming h...,2022-03-22,"[big, story, monday, oil, storm, high, 6, decl...",big story monday oil storm high 6 decline last...
1,MO,By Peter NurseInvesting.com -- Stocks in focu...,2022-03-22,"[peter, nurseinvesting.com, stocks, focus, pre...",peter nurseinvesting.com stocks focus premarke...
2,MO,The members of this alcohol and tobacco indus...,2022-03-17,"[member, alcohol, tobacco, industrybacked, mar...",member alcohol tobacco industrybacked marijuan...
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24,"[reuters, administrative, law, judge, dismiss,...",reuters administrative law judge dismiss feder...
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15,"[washington, reuters, judge, u.s., federal, tr...",washington reuters judge u.s. federal trade co...
...,...,...,...,...,...
6365,AAPL,By Stephen Nellis(Reuters) - If Nvidia Corp ...,2020-10-07,"[stephen, nellis, reuters, nvidia, corp, nvda,...",stephen nellis reuters nvidia corp nvda buy ar...
6366,AAPL,By Yasin EbrahimInvesting.com – The Dow ralli...,2020-10-07,"[yasin, ebrahiminvesting.com, dow, rally, wedn...",yasin ebrahiminvesting.com dow rally wednesday...
6367,AAPL,By Geoffrey Smith Investing.com -- Donald Tru...,2020-10-06,"[geoffrey, smith, investing.com, donald, trump...",geoffrey smith investing.com donald trump retu...
6368,AAPL,By Sruthi Shankar(Reuters) - European stocks ...,2020-10-06,"[sruthi, shankar, reuters, european, stock, ea...",sruthi shankar reuters european stock ease two...


In [10]:
def get_quarterly_dates(row):
    quarterly_dates = np.array(['03-31', '06-30', '09-30', '12-31'])
    current_date = str(row['Date'])
    year = current_date[:5]
    current_date = current_date[5:10]
    new_date = quarterly_dates[quarterly_dates >= current_date]
    new_date = min(new_date)
    return year + new_date

In [11]:
df['quarterlyDateEnding'] = df.apply(get_quarterly_dates, axis = 1)
df

,Symbol,Article,Date,title/no_stopwords,title/clean,quarterlyDateEnding
0,MO,The other big story Monday was oil storming h...,2022-03-22,"[big, story, monday, oil, storm, high, 6, decl...",big story monday oil storm high 6 decline last...,2022-03-31
1,MO,By Peter NurseInvesting.com -- Stocks in focu...,2022-03-22,"[peter, nurseinvesting.com, stocks, focus, pre...",peter nurseinvesting.com stocks focus premarke...,2022-03-31
2,MO,The members of this alcohol and tobacco indus...,2022-03-17,"[member, alcohol, tobacco, industrybacked, mar...",member alcohol tobacco industrybacked marijuan...,2022-03-31
3,MO,(Reuters) - An administrative law judge dismi...,2022-02-24,"[reuters, administrative, law, judge, dismiss,...",reuters administrative law judge dismiss feder...,2022-03-31
4,MO,WASHINGTON (Reuters) -A judge at the U.S. Fed...,2022-02-15,"[washington, reuters, judge, u.s., federal, tr...",washington reuters judge u.s. federal trade co...,2022-03-31
...,...,...,...,...,...,...
6365,AAPL,By Stephen Nellis(Reuters) - If Nvidia Corp ...,2020-10-07,"[stephen, nellis, reuters, nvidia, corp, nvda,...",stephen nellis reuters nvidia corp nvda buy ar...,2020-12-31
6366,AAPL,By Yasin EbrahimInvesting.com – The Dow ralli...,2020-10-07,"[yasin, ebrahiminvesting.com, dow, rally, wedn...",yasin ebrahiminvesting.com dow rally wednesday...,2020-12-31
6367,AAPL,By Geoffrey Smith Investing.com -- Donald Tru...,2020-10-06,"[geoffrey, smith, investing.com, donald, trump...",geoffrey smith investing.com donald trump retu...,2020-12-31
6368,AAPL,By Sruthi Shankar(Reuters) - European stocks ...,2020-10-06,"[sruthi, shankar, reuters, european, stock, ea...",sruthi shankar reuters european stock ease two...,2020-12-31


In [12]:
# groupby symbol and quarterlyDateEnding to combine title into one document
new_df = df.groupby(['Symbol', 'quarterlyDateEnding'])['title/clean'].apply(" ".join).reset_index()
new_df

,Symbol,quarterlyDateEnding,title/clean
0,AAL,2015-03-31,jeffrey dastinnew york reuters united airlines...
1,AAL,2015-06-30,alwyn scottmiami reuters air traveler familiar...
2,AAL,2015-09-30,alwyn scottmobile ala. reuters airbus group pa...
3,AAL,2015-12-31,reuters united airlines wednesday say cancel f...
4,AAL,2016-03-31,jeffrey dastin reuters u.s. airline hop get li...
...,...,...,...
196,MO,2020-06-30,investing.com altria nyse mo stock fell 3.08 t...
197,MO,2020-09-30,merrill lynch analyst lisa lewandowski maintai...
198,MO,2021-06-30,tobacco company might affect biden administrat...
199,MO,2021-12-31,daniel shvartsmaninvesting.com anheuser busch ...


In [13]:
vectorizer = TfidfVectorizer(max_features = 1000, min_df = 10, max_df = .9, stop_words=stopwords.words('english'))
tfidf = vectorizer.fit_transform(new_df['title/clean'].tolist())
tfidf = tfidf.toarray()
tfidf

array([[0.        , 0.01155189, 0.        , ..., 0.058414  , 0.        ,
        0.02715764],
       [0.        , 0.01809864, 0.02313321, ..., 0.00871607, 0.00798999,
        0.06807773],
       [0.05003927, 0.03687814, 0.00673381, ..., 0.        , 0.        ,
        0.03467913],
       ...,
       [0.01019656, 0.04508822, 0.0192102 , ..., 0.        , 0.00995253,
        0.01413322],
       [0.        , 0.04617808, 0.        , ..., 0.        , 0.06115859,
        0.        ],
       [0.        , 0.02581577, 0.03299705, ..., 0.        , 0.0341906 ,
        0.0121382 ]])

In [14]:
tfidf_pd = pd.DataFrame(tfidf)
tfidf_pd['Symbol'] = new_df['Symbol']
tfidf_pd['quarterlyDateEnding'] = new_df['quarterlyDateEnding']
tfidf_pd

,0,1,2,3,4,5,6,7,8,9,...,992,993,994,995,996,997,998,999,Symbol,quarterlyDateEnding
0,0.000000,0.011552,0.000000,0.000000,0.011316,0.024087,0.012837,0.006631,0.000000,0.009727,...,0.000000,0.060977,0.008739,0.000000,0.075541,0.058414,0.000000,0.027158,AAL,2015-03-31
1,0.000000,0.018099,0.023133,0.000000,0.023639,0.025158,0.033519,0.020778,0.044964,0.000000,...,0.000000,0.023160,0.000000,0.000000,0.089421,0.008716,0.007990,0.068078,AAL,2015-06-30
2,0.050039,0.036878,0.006734,0.000000,0.015482,0.043939,0.035125,0.012097,0.028047,0.008872,...,0.006734,0.020225,0.000000,0.000000,0.105649,0.000000,0.000000,0.034679,AAL,2015-09-30
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.055916,0.000000,0.000000,0.050799,0.000000,0.000000,0.000000,AAL,2015-12-31
4,0.009724,0.050166,0.000000,0.000000,0.000000,0.007471,0.015927,0.008228,0.022892,0.012068,...,0.027480,0.068779,0.010843,0.000000,0.112474,0.031062,0.000000,0.040435,AAL,2016-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.000000,0.029169,0.000000,0.000000,0.028574,0.000000,0.000000,0.066975,0.031058,0.000000,...,0.111850,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MO,2020-06-30
197,0.000000,0.012270,0.015683,0.020663,0.012020,0.012792,0.027270,0.042261,0.065323,0.000000,...,0.094101,0.035328,0.037130,0.000000,0.064191,0.000000,0.065003,0.023077,MO,2020-09-30
198,0.010197,0.045088,0.019210,0.000000,0.036807,0.031338,0.033402,0.043136,0.040006,0.000000,...,0.067236,0.064909,0.000000,0.014319,0.085177,0.000000,0.009953,0.014133,MO,2021-06-30
199,0.000000,0.046178,0.000000,0.000000,0.000000,0.048143,0.025657,0.026507,0.000000,0.000000,...,0.088535,0.000000,0.000000,0.000000,0.080526,0.000000,0.061159,0.000000,MO,2021-12-31


In [15]:
prices = pd.read_csv("monthly_prices.csv")
prices.columns = ['Date', 'open', 'high', 'low', 'close', 'volume', 'Symbol']
prices

,Date,open,high,low,close,volume,Symbol
0,2022-01-21,178.32,181.780,172.5600,172.65,34039389,MMM
1,2021-12-31,172.55,179.180,169.1801,177.63,48251474,MMM
2,2021-11-30,178.53,186.300,169.9400,170.04,47872133,MMM
3,2021-10-29,176.25,183.820,173.7800,178.68,48241875,MMM
4,2021-09-30,194.55,195.606,175.3700,175.42,67366254,MMM
...,...,...,...,...,...,...,...
118000,2013-07-31,31.02,31.650,28.9700,29.81,157249700,ZTS
118001,2013-06-28,32.01,32.990,29.4000,30.89,292403400,ZTS
118002,2013-05-31,32.97,34.280,31.7500,32.00,72809400,ZTS
118003,2013-04-30,33.42,34.740,30.4200,33.02,42045100,ZTS


In [16]:
# add prices to new_tfidf
new_tfidf = tfidf_pd.copy()
new_tfidf['Price'] = np.nan
for i in new_tfidf.index:
    symbol = new_tfidf.loc[i]['Symbol']
    #print(symbol)
    date = new_tfidf.loc[i]['quarterlyDateEnding']
    year_month = date[:7]
    tmp = prices[prices['Symbol'] == symbol]
    new_tmp = tmp[tmp['Date'].str.contains(year_month)]
    #print(tmp['Date'].str.contains(year_month) == True)
    #print(len(new_tmp))
    if len(new_tmp) == 0:
        continue
    new_tfidf.at[i, 'Price'] = new_tmp.iloc[0]['close']
new_tfidf

,0,1,2,3,4,5,6,7,8,9,...,993,994,995,996,997,998,999,Symbol,quarterlyDateEnding,Price
0,0.000000,0.011552,0.000000,0.000000,0.011316,0.024087,0.012837,0.006631,0.000000,0.009727,...,0.060977,0.008739,0.000000,0.075541,0.058414,0.000000,0.027158,AAL,2015-03-31,52.780
1,0.000000,0.018099,0.023133,0.000000,0.023639,0.025158,0.033519,0.020778,0.044964,0.000000,...,0.023160,0.000000,0.000000,0.089421,0.008716,0.007990,0.068078,AAL,2015-06-30,39.935
2,0.050039,0.036878,0.006734,0.000000,0.015482,0.043939,0.035125,0.012097,0.028047,0.008872,...,0.020225,0.000000,0.000000,0.105649,0.000000,0.000000,0.034679,AAL,2015-09-30,38.830
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.055916,0.000000,0.000000,0.050799,0.000000,0.000000,0.000000,AAL,2015-12-31,42.350
4,0.009724,0.050166,0.000000,0.000000,0.000000,0.007471,0.015927,0.008228,0.022892,0.012068,...,0.068779,0.010843,0.000000,0.112474,0.031062,0.000000,0.040435,AAL,2016-03-31,41.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.000000,0.029169,0.000000,0.000000,0.028574,0.000000,0.000000,0.066975,0.031058,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MO,2020-06-30,39.250
197,0.000000,0.012270,0.015683,0.020663,0.012020,0.012792,0.027270,0.042261,0.065323,0.000000,...,0.035328,0.037130,0.000000,0.064191,0.000000,0.065003,0.023077,MO,2020-09-30,38.640
198,0.010197,0.045088,0.019210,0.000000,0.036807,0.031338,0.033402,0.043136,0.040006,0.000000,...,0.064909,0.000000,0.014319,0.085177,0.000000,0.009953,0.014133,MO,2021-06-30,47.680
199,0.000000,0.046178,0.000000,0.000000,0.000000,0.048143,0.025657,0.026507,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.080526,0.000000,0.061159,0.000000,MO,2021-12-31,47.390


In [17]:
nona_tfidf = new_tfidf.dropna()
nona_tfidf

,0,1,2,3,4,5,6,7,8,9,...,993,994,995,996,997,998,999,Symbol,quarterlyDateEnding,Price
0,0.000000,0.011552,0.000000,0.000000,0.011316,0.024087,0.012837,0.006631,0.000000,0.009727,...,0.060977,0.008739,0.000000,0.075541,0.058414,0.000000,0.027158,AAL,2015-03-31,52.780
1,0.000000,0.018099,0.023133,0.000000,0.023639,0.025158,0.033519,0.020778,0.044964,0.000000,...,0.023160,0.000000,0.000000,0.089421,0.008716,0.007990,0.068078,AAL,2015-06-30,39.935
2,0.050039,0.036878,0.006734,0.000000,0.015482,0.043939,0.035125,0.012097,0.028047,0.008872,...,0.020225,0.000000,0.000000,0.105649,0.000000,0.000000,0.034679,AAL,2015-09-30,38.830
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.055916,0.000000,0.000000,0.050799,0.000000,0.000000,0.000000,AAL,2015-12-31,42.350
4,0.009724,0.050166,0.000000,0.000000,0.000000,0.007471,0.015927,0.008228,0.022892,0.012068,...,0.068779,0.010843,0.000000,0.112474,0.031062,0.000000,0.040435,AAL,2016-03-31,41.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.000000,0.009656,0.000000,0.000000,0.000000,0.010067,0.042920,0.033257,0.010281,0.000000,...,0.000000,0.014610,0.000000,0.000000,0.000000,0.000000,0.000000,MO,2020-03-31,38.670
196,0.000000,0.029169,0.000000,0.000000,0.028574,0.000000,0.000000,0.066975,0.031058,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MO,2020-06-30,39.250
197,0.000000,0.012270,0.015683,0.020663,0.012020,0.012792,0.027270,0.042261,0.065323,0.000000,...,0.035328,0.037130,0.000000,0.064191,0.000000,0.065003,0.023077,MO,2020-09-30,38.640
198,0.010197,0.045088,0.019210,0.000000,0.036807,0.031338,0.033402,0.043136,0.040006,0.000000,...,0.064909,0.000000,0.014319,0.085177,0.000000,0.009953,0.014133,MO,2021-06-30,47.680


In [18]:
# add previous quarter prices to df
nona_tfidf['prevPrice'] = np.nan
d = {'-03' : '-12', '-06' : '-03', '-09' : '-06', '-12' : '-09'}
for i in nona_tfidf.index:
    symbol = new_tfidf.loc[i]['Symbol']
    #print(symbol)
    date = new_tfidf.loc[i]['quarterlyDateEnding']
    year_month = date[:7]
    month = date[4:7]
    year = date[:4]
    tmp = prices[prices['Symbol'] == symbol]
    new_tmp = tmp[tmp['Date'].str.contains(year + d[month])]
    if len(new_tmp) == 0:
        continue
    nona_tfidf.at[i, 'prevPrice'] = new_tmp.iloc[0]['close']
nona_tfidf = nona_tfidf.dropna()
nona_tfidf

/var/folders/qr/c62bp79j3c91gdkr6wydmq7h0000gn/T/ipykernel_35680/567317927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nona_tfidf['prevPrice'] = np.nan


,0,1,2,3,4,5,6,7,8,9,...,994,995,996,997,998,999,Symbol,quarterlyDateEnding,Price,prevPrice
0,0.000000,0.011552,0.000000,0.000000,0.011316,0.024087,0.012837,0.006631,0.000000,0.009727,...,0.008739,0.000000,0.075541,0.058414,0.000000,0.027158,AAL,2015-03-31,52.780,42.350
1,0.000000,0.018099,0.023133,0.000000,0.023639,0.025158,0.033519,0.020778,0.044964,0.000000,...,0.000000,0.000000,0.089421,0.008716,0.007990,0.068078,AAL,2015-06-30,39.935,52.780
2,0.050039,0.036878,0.006734,0.000000,0.015482,0.043939,0.035125,0.012097,0.028047,0.008872,...,0.000000,0.000000,0.105649,0.000000,0.000000,0.034679,AAL,2015-09-30,38.830,39.935
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.050799,0.000000,0.000000,0.000000,AAL,2015-12-31,42.350,38.830
4,0.009724,0.050166,0.000000,0.000000,0.000000,0.007471,0.015927,0.008228,0.022892,0.012068,...,0.010843,0.000000,0.112474,0.031062,0.000000,0.040435,AAL,2016-03-31,41.010,46.690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.000000,0.009656,0.000000,0.000000,0.000000,0.010067,0.042920,0.033257,0.010281,0.000000,...,0.014610,0.000000,0.000000,0.000000,0.000000,0.000000,MO,2020-03-31,38.670,41.000
196,0.000000,0.029169,0.000000,0.000000,0.028574,0.000000,0.000000,0.066975,0.031058,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MO,2020-06-30,39.250,38.670
197,0.000000,0.012270,0.015683,0.020663,0.012020,0.012792,0.027270,0.042261,0.065323,0.000000,...,0.037130,0.000000,0.064191,0.000000,0.065003,0.023077,MO,2020-09-30,38.640,39.250
198,0.010197,0.045088,0.019210,0.000000,0.036807,0.031338,0.033402,0.043136,0.040006,0.000000,...,0.000000,0.014319,0.085177,0.000000,0.009953,0.014133,MO,2021-06-30,47.680,51.160


In [19]:
date_split = '2018-09-30'
X_train = nona_tfidf[nona_tfidf['quarterlyDateEnding'] < date_split].drop(['quarterlyDateEnding', 'Symbol', 'Price'], axis = 1)
y_train = nona_tfidf[nona_tfidf['quarterlyDateEnding'] < date_split]['Price']
X_test = nona_tfidf[nona_tfidf['quarterlyDateEnding'] >= date_split].drop(['quarterlyDateEnding', 'Symbol', 'Price'], axis = 1)
y_test = nona_tfidf[nona_tfidf['quarterlyDateEnding'] >= date_split]['Price']
model = models.Sequential()
model.add(layers.Dense(18, activation='relu', input_shape=(1001, )))
model.add(layers.Dense(18, activation='relu'))
model.add(layers.Dense(9, activation='relu'))
model.add(layers.Dense(9, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit(X_train, y_train, epochs = 10, batch_size = 100)

2022-04-10 16:46:04.947565: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
1/1 [==============================] - 0s 428ms/step - loss: 180504.6094 - mean_absolute_error: 246.0559
Epoch 2/10
1/1 [==============================] - 0s 2ms/step - loss: 179833.5625 - mean_absolute_error: 245.5762
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 179379.0312 - mean_absolute_error: 245.2629
Epoch 4/10
1/1 [==============================] - 0s 2ms/step - loss: 179235.2500 - mean_absolute_error: 245.1637
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: 179025.5625 - mean_absolute_error: 245.0142
Epoch 6/10
1/1 [==============================] - 0s 3ms/step - loss: 178899.4375 - mean_absolute_error: 244.9211
Epoch 7/10
1/1 [==============================] - 0s 3ms/step - loss: 178761.4219 - mean_absolute_error: 244.8231
Epoch 8/10
1/1 [==============================] - 0s 2ms/step - loss: 178609.6406 - mean_absolute_error: 244.7129
Epoch 9/10
1/1 [==============================] - 0s 2ms/step - loss: 178439.0000 - me

In [21]:
# predict price using tf-idf two weeks after article
vectorizer = TfidfVectorizer(max_features = 1000, min_df = 10, max_df = .9, stop_words=stopwords.words('english'))
tfidf = vectorizer.fit_transform(df['title/clean'].tolist())
tfidf = tfidf.toarray()
tfidf

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.03015657, 0.        , ..., 0.        , 0.03836725,
        0.        ],
       [0.        , 0.        , 0.070518  , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.02420116, 0.0617626 , ..., 0.07049026, 0.        ,
        0.        ]])

In [29]:
tfidf_pd = pd.DataFrame(tfidf)
tfidf_pd['Symbol'] = df['Symbol']
tfidf_pd['currentDate'] = df['Date']
tfidf_pd['nextDate'] = df['Date'] + pd.DateOffset(days = 14)
tfidf_pd

,0,1,2,3,4,5,6,7,8,9,...,993,994,995,996,997,998,999,Symbol,currentDate,nextDate
0,0.0,0.000000,0.000000,0.0,0.0,0.092338,0.0,0.051718,0.000000,0.0,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-03-22,2022-04-05
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-03-22,2022-04-05
2,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00000,0.051065,0.00000,0.000000,0.0,MO,2022-03-17,2022-03-31
3,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-02-24,2022-03-10
4,0.0,0.000000,0.000000,0.0,0.0,0.075047,0.0,0.000000,0.000000,0.0,...,0.000000,0.096542,0.00000,0.039164,0.00000,0.000000,0.0,MO,2022-02-15,2022-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6359,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.098468,0.000000,0.00000,0.027120,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22
6360,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.07534,0.000000,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22
6361,0.0,0.030157,0.000000,0.0,0.0,0.000000,0.0,0.041223,0.036688,0.0,...,0.023243,0.000000,0.00000,0.000000,0.00000,0.038367,0.0,AAPL,2020-10-08,2020-10-22
6362,0.0,0.000000,0.070518,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00000,0.035194,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22


In [28]:
daily_prices = pd.read_csv('daily_prices.csv')
daily_prices

,Date,Open,High,Low,Close,Volume,Symbol
0,2014-04-09,107.283329,107.974706,106.885983,107.950867,2671900.0,MMM
1,2014-04-10,107.863458,108.419740,106.409171,106.464806,2857200.0,MMM
2,2014-04-11,105.988008,106.361514,105.209213,105.209213,2368200.0,MMM
3,2014-04-14,106.051558,106.107193,104.915156,105.813148,2058100.0,MMM
4,2014-04-15,105.908540,106.766808,104.915176,106.560181,2119000.0,MMM
...,...,...,...,...,...,...,...
1000210,2022-04-04,191.139999,191.979996,187.179993,188.740005,2603600.0,ZTS
1000211,2022-04-05,188.740005,190.740005,187.690002,188.399994,2059300.0,ZTS
1000212,2022-04-06,188.160004,192.330002,186.520004,191.380005,2843500.0,ZTS
1000213,2022-04-07,191.199997,199.339996,190.970001,197.929993,2781400.0,ZTS


In [31]:
current_price = []
for i in range(len(tfidf_pd)):
    if i % 500 == 0:
        print(i)
    if i % 2000 == 0:
        clear_output(wait = True)
    subset = daily_prices[(daily_prices['Date'] <= str(tfidf_pd.iloc[i]['currentDate'])[0: 10]) & (tfidf_pd.iloc[i]['Symbol'] == daily_prices['Symbol'])]
    if len(subset) == 0:
        current_price.append(np.nan)
        continue
    subset = subset[subset['Date'] == max(subset['Date'])]
    current_price.append(subset.iloc[0]['Close'])
current_price[0]

52.40848159790039

In [32]:
tfidf_pd['currentPrice'] = current_price
tfidf_pd

,0,1,2,3,4,5,6,7,8,9,...,994,995,996,997,998,999,Symbol,currentDate,nextDate,currentPrice
0,0.0,0.000000,0.000000,0.0,0.0,0.092338,0.0,0.051718,0.000000,0.0,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-03-22,2022-04-05,52.408482
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-03-22,2022-04-05,52.408482
2,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.00000,0.051065,0.00000,0.000000,0.0,MO,2022-03-17,2022-03-31,50.815872
3,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-02-24,2022-03-10,48.731728
4,0.0,0.000000,0.000000,0.0,0.0,0.075047,0.0,0.000000,0.000000,0.0,...,0.096542,0.00000,0.039164,0.00000,0.000000,0.0,MO,2022-02-15,2022-03-01,49.469044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6359,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.00000,0.027120,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22,113.923637
6360,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.07534,0.000000,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22,113.923637
6361,0.0,0.030157,0.000000,0.0,0.0,0.000000,0.0,0.041223,0.036688,0.0,...,0.000000,0.00000,0.000000,0.00000,0.038367,0.0,AAPL,2020-10-08,2020-10-22,113.923637
6362,0.0,0.000000,0.070518,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.00000,0.035194,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22,113.923637


In [34]:
# two weeks from current date
next_price = []
for i in range(len(tfidf_pd)):
    if i % 500 == 0:
        print(i)
    if i % 2000 == 0:
        clear_output(wait = True)
    next_date = tfidf_pd.iloc[i]['nextDate']
    next_date = str(next_date)[0:10]
    subset = daily_prices[(daily_prices['Date'] <= next_date) & (tfidf_pd.iloc[i]['Symbol'] == daily_prices['Symbol'])]
    if len(subset) == 0:
        next_price.append(np.nan)
        continue
    subset = subset[subset['Date'] == max(subset['Date'])]
    if subset.iloc[0]['Date'] <= str(tfidf_pd.iloc[i]['currentDate'] + pd.DateOffset(days = 7))[0:10]:
        next_price.append(np.nan)
        continue
    next_price.append(subset.iloc[0]['Close'])
next_price[0]

52.63999938964844

In [35]:
tfidf_pd['nextPrice'] = next_price
tfidf_pd

,0,1,2,3,4,5,6,7,8,9,...,995,996,997,998,999,Symbol,currentDate,nextDate,currentPrice,nextPrice
0,0.0,0.000000,0.000000,0.0,0.0,0.092338,0.0,0.051718,0.000000,0.0,...,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-03-22,2022-04-05,52.408482,52.639999
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-03-22,2022-04-05,52.408482,52.639999
2,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.051065,0.00000,0.000000,0.0,MO,2022-03-17,2022-03-31,50.815872,52.250000
3,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-02-24,2022-03-10,48.731728,50.363655
4,0.0,0.000000,0.000000,0.0,0.0,0.075047,0.0,0.000000,0.000000,0.0,...,0.00000,0.039164,0.00000,0.000000,0.0,MO,2022-02-15,2022-03-01,49.469044,50.697903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6359,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.027120,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22,113.923637,114.696533
6360,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.07534,0.000000,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22,113.923637,114.696533
6361,0.0,0.030157,0.000000,0.0,0.0,0.000000,0.0,0.041223,0.036688,0.0,...,0.00000,0.000000,0.00000,0.038367,0.0,AAPL,2020-10-08,2020-10-22,113.923637,114.696533
6362,0.0,0.000000,0.070518,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.035194,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22,113.923637,114.696533


In [37]:
tfidf_pd = tfidf_pd.dropna()
tfidf_pd

,0,1,2,3,4,5,6,7,8,9,...,995,996,997,998,999,Symbol,currentDate,nextDate,currentPrice,nextPrice
0,0.0,0.000000,0.000000,0.0,0.0,0.092338,0.0,0.051718,0.000000,0.0,...,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-03-22,2022-04-05,52.408482,52.639999
1,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-03-22,2022-04-05,52.408482,52.639999
2,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.051065,0.00000,0.000000,0.0,MO,2022-03-17,2022-03-31,50.815872,52.250000
3,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.000000,0.00000,0.000000,0.0,MO,2022-02-24,2022-03-10,48.731728,50.363655
4,0.0,0.000000,0.000000,0.0,0.0,0.075047,0.0,0.000000,0.000000,0.0,...,0.00000,0.039164,0.00000,0.000000,0.0,MO,2022-02-15,2022-03-01,49.469044,50.697903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6359,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.027120,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22,113.923637,114.696533
6360,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.07534,0.000000,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22,113.923637,114.696533
6361,0.0,0.030157,0.000000,0.0,0.0,0.000000,0.0,0.041223,0.036688,0.0,...,0.00000,0.000000,0.00000,0.038367,0.0,AAPL,2020-10-08,2020-10-22,113.923637,114.696533
6362,0.0,0.000000,0.070518,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.035194,0.00000,0.000000,0.0,AAPL,2020-10-08,2020-10-22,113.923637,114.696533


In [44]:
tfidf_pd[tfidf_pd['currentDate'] < '2020-03-30']

,0,1,2,3,4,5,6,7,8,9,...,995,996,997,998,999,Symbol,currentDate,nextDate,currentPrice,nextPrice
32,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.099649,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,MO,2020-03-27,2020-04-10,31.516800,35.189751
33,0.0,0.000000,0.0,0.000000,0.0,0.0,0.105511,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,MO,2020-03-27,2020-04-10,31.516800,35.189751
34,0.0,0.000000,0.0,0.000000,0.0,0.0,0.112109,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,MO,2020-03-20,2020-04-03,28.697470,32.179134
35,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,MO,2020-03-20,2020-04-03,28.697470,32.179134
36,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,MO,2020-03-19,2020-04-02,31.058229,31.155531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242,0.0,0.000000,0.0,0.069109,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.061958,0.025859,0.0,0.154985,0.035681,APA,2015-09-23,2015-10-07,32.691380,40.287479
5243,0.0,0.000000,0.0,0.072946,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.065398,0.027295,0.0,0.163589,0.037662,APA,2015-09-23,2015-10-07,32.691380,40.287479
5244,0.0,0.059482,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.113642,0.0,0.000000,0.052268,APA,2015-09-22,2015-10-06,33.884293,40.454140
5245,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.112020,0.0,0.000000,0.000000,APA,2015-09-16,2015-09-30,37.752514,34.349182


In [103]:
date_split = '2020-03-30'
X_train = tfidf_pd[tfidf_pd['currentDate'] < date_split].drop(['currentDate', 'Symbol', 'nextPrice', 'nextDate'], axis = 1)
y_train = tfidf_pd[tfidf_pd['currentDate'] < date_split]['nextPrice']
X_test = tfidf_pd[tfidf_pd['currentDate'] >= date_split].drop(['currentDate', 'Symbol', 'nextPrice', 'nextDate'], axis = 1)
y_test = tfidf_pd[tfidf_pd['currentDate'] >= date_split]['nextPrice']
model = models.Sequential()
model.add(layers.Dense(18, activation='relu', input_shape=(1001, )))
model.add(layers.Dense(18, activation='relu'))
model.add(layers.Dense(9, activation='relu'))
model.add(layers.Dense(9, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit(X_train, y_train, epochs = 10, batch_size = 100)

Epoch 1/10
32/32 [==============================] - 1s 2ms/step - loss: 1625716.8750 - mean_absolute_error: 1093.5776
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 1229277.1250 - mean_absolute_error: 948.3257
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 433090.0625 - mean_absolute_error: 534.9563
Epoch 4/10
32/32 [==============================] - 0s 1ms/step - loss: 15226.2979 - mean_absolute_error: 84.0459
Epoch 5/10
32/32 [==============================] - 0s 1ms/step - loss: 6087.6611 - mean_absolute_error: 51.8551
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 5580.4258 - mean_absolute_error: 50.0242
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 5578.5752 - mean_absolute_error: 49.9062
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 5575.8135 - mean_absolute_error: 49.9733
Epoch 9/10
32/32 [==============================] - 0s 1ms/step - loss: 5572.5225 -

In [104]:
# get MAE of neural network
model.evaluate(X_test, y_test)

100/100 [==============================] - 0s 691us/step - loss: 14466.8027 - mean_absolute_error: 68.4212


[14466.802734375, 68.42115020751953]

In [105]:
model.predict(X_test)

array([[ 59.628468],
       [ 60.732956],
       [ 58.191673],
       ...,
       [124.39891 ],
       [122.73229 ],
       [125.48888 ]], dtype=float32)

In [106]:
# invest 10000 evenly across top projected companies equally
init_investment = 10000
predicted = pd.DataFrame(tfidf_pd.loc[X_test.index]['currentPrice'])
predicted['predictedPrice'] = model.predict(X_test)
predicted['percentChange'] = ((predicted['predictedPrice'] - predicted['currentPrice']) / predicted['currentPrice']) * 100
predicted['actualPrice'] = y_test
predicted['articleDate'] = tfidf_pd.loc[X_test.index]['currentDate']
predicted

,currentPrice,predictedPrice,percentChange,actualPrice,articleDate
0,52.408482,59.628468,13.776369,52.639999,2022-03-22
1,52.408482,60.732956,15.883830,52.639999,2022-03-22
2,50.815872,58.191673,14.514758,52.250000,2022-03-17
3,48.731728,55.687737,14.274086,50.363655,2022-02-24
4,49.469044,57.274517,15.778501,50.697903,2022-02-15
...,...,...,...,...,...
6359,113.923637,122.024925,7.111156,114.696533,2020-10-08
6360,113.923637,122.492493,7.521578,114.696533,2020-10-08
6361,113.923637,124.398911,9.194995,114.696533,2020-10-08
6362,113.923637,122.732292,7.732069,114.696533,2020-10-08


In [95]:
predicted[predicted['percentChange'] > 10]

,currentPrice,predictedPrice,percentChange,actualPrice
0,52.408482,59.961105,14.411071,52.639999
1,52.408482,61.270489,16.909490,52.639999
2,50.815872,58.838551,15.787741,52.250000
3,48.731728,56.076820,15.072506,50.363655
4,49.469044,57.985161,17.215043,50.697903
...,...,...,...,...
6299,107.780052,119.049835,10.456279,119.410782
6302,107.780052,119.045769,10.452506,119.410782
6303,107.780052,119.983269,11.322333,119.410782
6323,110.187943,121.900887,10.629969,118.606773


In [96]:
tfidf_pd[tfidf_pd['currentDate'] < '2019-06-30']

,0,1,2,3,4,5,6,7,8,9,...,995,996,997,998,999,Symbol,currentDate,nextDate,currentPrice,nextPrice
71,0.051829,0.000000,0.000000,0.000000,0.035613,0.000000,0.153680,0.0,0.071202,0.0,...,0.000000,0.000000,0.0,0.000000,0.051427,MO,2019-06-21,2019-07-05,38.781551,39.411758
72,0.000000,0.000000,0.045084,0.060132,0.000000,0.043116,0.092775,0.0,0.042984,0.0,...,0.000000,0.022500,0.0,0.044951,0.031046,MO,2019-06-21,2019-07-05,38.781551,39.411758
73,0.000000,0.000000,0.000000,0.000000,0.099338,0.099608,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.071725,MO,2019-06-21,2019-07-05,38.781551,39.411758
74,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.069047,MO,2019-06-05,2019-06-19,39.524433,40.316658
75,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.046788,MO,2019-05-30,2019-06-13,39.938053,41.375072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5242,0.000000,0.000000,0.000000,0.069109,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.061958,0.025859,0.0,0.154985,0.035681,APA,2015-09-23,2015-10-07,32.691380,40.287479
5243,0.000000,0.000000,0.000000,0.072946,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.065398,0.027295,0.0,0.163589,0.037662,APA,2015-09-23,2015-10-07,32.691380,40.287479
5244,0.000000,0.059482,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.113642,0.0,0.000000,0.052268,APA,2015-09-22,2015-10-06,33.884293,40.454140
5245,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.112020,0.0,0.000000,0.000000,APA,2015-09-16,2015-09-30,37.752514,34.349182


In [97]:
pca = PCA(n_components=10)
reduction = pca.fit_transform(tfidf_pd.drop(['currentDate', 'Symbol', 'nextPrice', 'nextDate', 'currentPrice'], axis = 1))
df_reduction = pd.DataFrame(reduction)
df_reduction['currentDate'] = tfidf_pd['currentDate']
df_reduction['nextPrice'] = tfidf_pd['nextPrice']
df_reduction['currentPrice'] = tfidf_pd['currentPrice']
df_reduction

,0,1,2,3,4,5,6,7,8,9,currentDate,nextPrice,currentPrice
0,-0.102964,-0.030693,0.026835,0.083108,0.018470,0.167288,0.048995,0.026053,0.048460,-0.111973,2022-03-22,52.639999,52.408482
1,-0.232424,-0.007437,-0.073405,0.103845,0.258549,0.023886,0.040210,-0.035400,-0.159474,-0.056966,2022-03-22,52.639999,52.408482
2,0.064055,-0.054090,0.019946,-0.133403,0.003390,0.049672,-0.000438,-0.035995,0.057899,-0.037022,2022-03-17,52.250000,50.815872
3,-0.074745,-0.150976,-0.160349,0.117544,0.352167,-0.100146,0.052530,-0.128612,-0.110570,-0.002769,2022-02-24,50.363655,48.731728
4,0.003138,-0.123089,-0.070385,0.075254,0.260578,-0.049995,0.006645,-0.100695,-0.043699,-0.025982,2022-02-15,50.697903,49.469044
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6326,0.117238,-0.004798,-0.054161,-0.070608,0.003367,0.059008,0.092757,-0.069112,0.119414,-0.022180,2020-10-27,115.112785,115.538788
6327,-0.250671,0.097328,-0.083777,-0.122010,-0.137766,0.076527,0.224122,0.025253,0.060631,-0.047189,2020-10-24,117.812691,113.992989
6328,-0.164180,0.093233,0.010047,-0.190694,-0.081823,-0.063267,-0.102834,0.053791,-0.072674,-0.007078,2020-10-23,117.812691,113.992989
6329,-0.110775,-0.005430,-0.032796,-0.109278,-0.076727,0.006635,0.095786,0.011690,0.084295,-0.036365,2020-10-23,117.812691,113.992989


In [98]:
pca.explained_variance_ratio_

array([0.04341896, 0.02131278, 0.0200541 , 0.01815017, 0.01622648,
       0.01367886, 0.01281812, 0.0124385 , 0.0108131 , 0.01022378])

In [108]:
date_split = '2020-03-30'
X_train = df_reduction[df_reduction['currentDate'] < date_split].drop(['currentDate', 'nextPrice',], axis = 1)
y_train = df_reduction[df_reduction['currentDate'] < date_split]['nextPrice']
X_test = df_reduction[df_reduction['currentDate'] >= date_split].drop(['currentDate', 'nextPrice'], axis = 1)
y_test = df_reduction[df_reduction['currentDate'] >= date_split]['nextPrice']
model = models.Sequential()
model.add(layers.Dense(18, activation='relu', input_shape=(11, )))
model.add(layers.Dense(18, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit(X_train, y_train, epochs = 10, batch_size = 100)

Epoch 1/10
32/32 [==============================] - 0s 1ms/step - loss: 1461905.8750 - mean_absolute_error: 1035.8011
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 835491.1250 - mean_absolute_error: 778.5145
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 232112.3125 - mean_absolute_error: 389.9300
Epoch 4/10
32/32 [==============================] - 0s 1ms/step - loss: 10450.1934 - mean_absolute_error: 68.1206
Epoch 5/10
32/32 [==============================] - 0s 939us/step - loss: 5860.3789 - mean_absolute_error: 49.5392
Epoch 6/10
32/32 [==============================] - 0s 970us/step - loss: 5607.3076 - mean_absolute_error: 49.2000
Epoch 7/10
32/32 [==============================] - 0s 985us/step - loss: 5609.0347 - mean_absolute_error: 49.0322
Epoch 8/10
32/32 [==============================] - 0s 948us/step - loss: 5609.7285 - mean_absolute_error: 49.0533
Epoch 9/10
32/32 [==============================] - 0s 946us/step - loss: 56

In [109]:
predicted = pd.DataFrame(tfidf_pd.loc[X_test.index]['currentPrice'])
predicted['predictedPrice'] = model.predict(X_test)
predicted['percentChange'] = ((predicted['predictedPrice'] - predicted['currentPrice']) / predicted['currentPrice']) * 100
predicted['actualPrice'] = y_test
predicted['articleDate'] = tfidf_pd.loc[X_test.index]['currentDate']
predicted

,currentPrice,predictedPrice,percentChange,actualPrice,articleDate
0,52.408482,53.423561,1.936861,52.639999,2022-03-22
1,52.408482,53.466808,2.019381,52.639999,2022-03-22
2,50.815872,51.929867,2.192218,52.250000,2022-03-17
3,48.731728,49.865898,2.327376,50.363655,2022-02-24
4,49.469044,50.629959,2.346751,50.697903,2022-02-15
...,...,...,...,...,...
6326,115.538788,116.833870,1.120907,115.112785,2020-10-27
6327,113.992989,115.172928,1.035098,117.812691,2020-10-24
6328,113.992989,115.505302,1.326673,117.812691,2020-10-23
6329,113.992989,115.335983,1.178138,117.812691,2020-10-23


In [110]:
model = DecisionTreeRegressor()
date_split = '2020-03-30'
X_train = tfidf_pd[tfidf_pd['currentDate'] < date_split].drop(['currentDate', 'Symbol', 'nextPrice', 'nextDate'], axis = 1)
y_train = tfidf_pd[tfidf_pd['currentDate'] < date_split]['nextPrice']
X_test = tfidf_pd[tfidf_pd['currentDate'] >= date_split].drop(['currentDate', 'Symbol', 'nextPrice', 'nextDate'], axis = 1)
y_test = tfidf_pd[tfidf_pd['currentDate'] >= date_split]['nextPrice']
model.fit(X_train, y_train)

/Users/ryanshintaku/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


DecisionTreeRegressor()

In [111]:
predicted = pd.DataFrame(tfidf_pd.loc[X_test.index]['currentPrice'])
predicted['predictedPrice'] = model.predict(X_test)
predicted['percentChange'] = ((predicted['predictedPrice'] - predicted['currentPrice']) / predicted['currentPrice']) * 100
predicted['actualPrice'] = y_test
predicted['articleDate'] = tfidf_pd.loc[X_test.index]['currentDate']
predicted

/Users/ryanshintaku/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,currentPrice,predictedPrice,percentChange,actualPrice,articleDate
0,52.408482,54.410156,3.819372,52.639999,2022-03-22
1,52.408482,54.410156,3.819372,52.639999,2022-03-22
2,50.815872,45.619766,-10.225360,52.250000,2022-03-17
3,48.731728,54.599979,12.041953,50.363655,2022-02-24
4,49.469044,54.599979,10.372013,50.697903,2022-02-15
...,...,...,...,...,...
6359,113.923637,111.459747,-2.162756,114.696533,2020-10-08
6360,113.923637,109.760254,-3.654539,114.696533,2020-10-08
6361,113.923637,109.760254,-3.654539,114.696533,2020-10-08
6362,113.923637,109.760254,-3.654539,114.696533,2020-10-08
